# OpenEO: LAI

In [ ]:
import openeo
import geopandas as gpd
from pathlib import Path
import numpy as np
import xarray as xr
import hvplot.xarray
import matplotlib.pyplot as plt
import pickle
import os
connection = openeo.connect("openeo.vito.be").authenticate_oidc()
pad = Path(os.getcwd())
if pad.name != "Python":
    pad_correct = Path("../../Python")
    os.chdir(pad_correct)
from functions.pre_processing import pre_processing_pipeline

#set all to True to process 
job_excec = False
download_exec = False


In [ ]:
shape_zwalm = gpd.read_file('data/Zwalm_shape/zwalm_shapefile_emma.shp')
shape_zwalm.plot()
extent = shape_zwalm.total_bounds
print(extent)

In [ ]:
#connection.list_collections()

In [ ]:
#connection.describe_collection("PROBAV_L3_S5_TOC_100M")
connection.describe_collection('CGLS_LAI300_V1_GLOBAL')

https://land.copernicus.eu/global/products/lai belangrijk: The physical values (PV) are derived from the digital number (DN) using the relation: PV = Scaling * DN + Offset. Some specific values are used: 255 for missing pixels.

In [ ]:
collection = "CGLS_LAI300_V1_GLOBAL"
spatial_extent = {'west':extent[0],'east':extent[2],'south':extent[1],'north':extent[3]}
temporal_extent = ["2014-11-18", "2022-11-05"] 
#bands = ["NDVI"]

In [ ]:
list_temp_extent = []
list_temp_extent.append([temporal_extent[0],"2014-12-31"])
import numpy as np
for year in np.arange(2015,2023):
    if year == 2022:
        #print([str(year)+"-01-01",temporal_extent[1]])
        list_temp_extent.append([str(year)+"-01-01",temporal_extent[1]])
    else:
        #print([str(year)+"-01-01",str(year)+ "-12-31"])
        list_temp_extent.append([str(year)+"-01-01",str(year)+ "-12-31"])
print(list_temp_extent)

In [ ]:
years = np.arange(2014,2023)
job_title_list = []
job_id_list = []
if job_excec:
    for i, temporal_extent in enumerate(list_temp_extent):
        probav = connection.load_collection(
            collection_id = collection,
            spatial_extent= spatial_extent,
            temporal_extent = temporal_extent#,
            #bands = bands
        )
        probav = probav.mask_polygon(shape_zwalm['geometry'].values[0])
        job_title = "probav-" +  str(years[i])
        job_title_list.append(job_title)
        job_probav = probav.create_job(title = job_title, out_format= 'NetCDF')
        job_probav_id = job_probav.job_id
        if job_probav_id:
            print("Batch job created with id: ",job_probav_id)
            job_probav.start_job()
            job_id_list.append(job_probav_id)
        else:
            print("Error! Job ID is None")

In [ ]:
# job_id_list
# with open('temp/probva_job_id_list.pkl', 'wb') as handle:
#     pickle.dump(job_id_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# job_id_list = pickle.load(open("temp/probva_job_id_list.pkl", "rb"))
if download_exec:
    for i,job_id in enumerate(job_id_list):
        job_connection = connection.job(job_id)
        results = job_connection.get_results()
        name_netcdf = job_title_list[i] + '.nc'
        filepath = "data/LAI/" + name_netcdf
        print(filepath)
        results.download_file(filepath)

In [ ]:
job_id_list

## Check dataset out

In [ ]:
LAI_xr = xr.open_mfdataset('data/LAI/*.nc', decode_coords = 'all')
LAI_xr

uint 8 => 0-255 only! PV = Scaling * DN + Offset. https://land.copernicus.eu/global/products/lai  
scaling = 1/30
offset = 0

In [ ]:
LAI_xr_masked = LAI_xr.where(LAI_xr['LAI'] != 255) #255 = reserverd for No value!
LAI_xr_masked['LAI'] = LAI_xr_masked['LAI'].astype(np.float32)

In [ ]:
scaling = 1/30
offset = 0
LAI_xr_masked['LAI_pv'] = LAI_xr_masked['LAI']*scaling + offset
LAI_xr_masked

In [ ]:
LAI_xr_masked['LAI'].hvplot.image('x','y',geo = True, frame_width = 350, tiles = 'OSM',cmap = 'cividis')

Give flag to dates without full data availability

In [ ]:
#determine how many pixels for +- full image
xr_shape = LAI_xr_masked['LAI'].shape
nr_pixels = xr_shape[1]*xr_shape[2]
print('Total number of pixels: ' + str(nr_pixels))
nr_nan_full = np.sum(np.isnan(LAI_xr_masked['LAI'].isel(t=0))).values
print('Numer of nan pixels full image: ' + str(nr_nan_full))
nan_cutoff = nr_nan_full/nr_pixels
print('Percentage nan pixels full imgage: ' + str(nan_cutoff*100) + '%')
nan_cutoff = nan_cutoff + 0.05 #add 5% margin before classifying as not full

In [ ]:
nr_timestemps = xr_shape[0]
bool_full_image = []
for i in range(nr_timestemps):
    LAI_ds = LAI_xr_masked['LAI'].isel(t=i)
    temp_nancount = np.sum(np.isnan(LAI_ds)).values
    nan_frac = temp_nancount/nr_pixels
    if nan_frac > nan_cutoff:
        bool_full_image.append(0)
    else:
        bool_full_image.append(1)
pos_full = np.where(bool_full_image)[0].tolist()

In [ ]:
da = xr.DataArray(
    data = bool_full_image,
    dims = ['t'],
    coords = dict(t = LAI_xr_masked['t'].values)
)
da = da.astype(np.int8)
LAI_xr_masked['bool_full_image'] = da
LAI_xr_masked

seperate xarray with only full images

In [ ]:
LAI_xr_masked_full = LAI_xr_masked.isel(t = pos_full)
LAI_xr_masked_full

In [ ]:
LAI_xr_masked.to_netcdf('data/LAI/LAI_cube_Zwalm.nc', mode = 'w')

## Average trends

In [ ]:
average_trends = LAI_xr_masked['LAI_pv'].mean(dim =['x','y'])
average_trends_full = LAI_xr_masked_full['LAI_pv'].mean(dim = ['x','y'])

In [ ]:
display(average_trends)
fig, (ax, ax2) = plt.subplots(1,2, figsize = (14,8))
average_trends.plot(ax = ax, marker = 'o')
LAI_xr_masked['bool_full_image'].plot(ax = ax)
ax.set_ylim([0,4])
ax.set_title('All timesteps included')

average_trends_full.plot(ax = ax2, marker = 'o')
LAI_xr_masked_full['bool_full_image'].plot(ax = ax2)
ax2.set_ylim([0,4])
ax2.set_title('Only timestep with full images included')

Preference for only full images, better trend?